In [ ]:
from PIL import Image
import numpy as np
import cv2
from matplotlib import pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (20, 20)

In [46]:
from datetime import datetime
from shutil import rmtree
import os
from glob import glob
from PIL import ImageDraw, ImageFont

In [329]:
names = sorted(glob("/home/gmo/Downloads/fake_data/res/name/*.png"))
labels = names = sorted(glob("/home/gmo/Downloads/fake_data/res/name/*.txt"))

In [342]:
names = sorted(glob("all/*.png"))
for name in names:
    new_path = name.split('.')[0]+".txt"
    text = name.split('_')[-1].split('.')[0]
    with open(new_path, "w") as f:
        f.write(text)

In [318]:
path = '02_02'
rows_num = 24

In [319]:
img = Image.open(f"{path}.jpg")
gray = np.array(img.convert('L'))
gray = cv2.bilateralFilter(gray, 11, 17, 17)
# _, thresh = cv2.threshold(gray,0,255,cv2.THRESH_OTSU)
edged = cv2.Canny(gray, 30, 200) 
cnts = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]

def check_box(c):
    x,y,w,h = cv2.boundingRect(c)
    area = cv2.contourArea(c)
    return w > 150 and h > 20
#     return True
cnts = [c for c in cnts if check_box(c)]
cnts = sorted(cnts, key=lambda cnts: cv2.boundingRect(cnts)[1], reverse=False)
# plt.imshow(edged)
# plt.imshow(cv2.drawContours(np.array(img), cnts, -1, (0, 255, 0), 3) )

In [320]:
def drawRect(img, box):
    return cv2.rectangle(img, set(box[0:2]), set(box[2:4]))

def rect(c):
    x, y, w, h = cv2.boundingRect(c)
    return x, y, x+w, y+h

boxes = np.array(list(map(rect, cnts)), dtype=float)
threshed = non_max_suppression_fast(boxes, 0.1)
threshed[0], threshed[1], len(threshed)

(array([  21,  997,  348, 1034]), array([ 390,  998,  722, 1033]), 96)

In [321]:
def find_table_in_boxes(boxes, threshold=(10, 10), columns=4):
    rows = {}
    cols = {}

    # Clustering the bounding boxes by their positions
    for box in boxes:
        x, y, x_2, y_2 = box
        x, y, w, h = x, y, x_2-x, y_2-y

#         (x, y, w, h) = box
        col_key = x // threshold[0]
        row_key = y // threshold[1]
        cols[row_key] = [box] if col_key not in cols else cols[col_key] + [box]
        rows[row_key] = [box] if row_key not in rows else rows[row_key] + [box]

    # Filtering out the clusters having less than 2 cols
    table_cells = list(filter(lambda r: len(r) == columns, rows.values()))
    # Sorting the row cells by x coord
    table_cells = [list(sorted(tb, key=lambda r: r[0])) for tb in table_cells]
    # Sorting rows by the y coord
    table_cells = list(sorted(table_cells, key=lambda r: r[0][1]))

    return table_cells

In [322]:
ls = []
best = None
for n in range(5, 50):
    k = np.array(img)
    num = len(np.array(find_table_in_boxes(threshed, (n, n), 4)).reshape(-1, 4))
    if num==rows_num*4:
        print(n, len(np.array(find_table_in_boxes(threshed, (n, n), 4)).reshape(-1, 4)))
        best = n
        break

k = np.array(img)
table = np.array(find_table_in_boxes(threshed, (best, best), 4)).reshape(-1, 4)
# draw_bounding_boxes_on_image_array(k, table, thickness=4)
# plt.imshow(k)

33 96


In [323]:
# table_1 = table
# img_1 = img
table_2 = table
img_2 = img

In [324]:
tables = []
tables.extend(table_1)
tables.extend(table_2)

In [325]:
len(tables)/4

49.0

In [326]:
ls = []

np_img = np.array(img_1)
nc = 0
for b in tables[0:25*4]:
    x, y, x2, y2 = b
    ROI = np_img[y:y2, x:x2]
    ls.append(ROI.copy())
    nc += 1
print(nc)

np_img = np.array(img_2)
nc = 0
for b in tables[25*4:]:
    x, y, x2, y2 = b
    ROI = np_img[y:y2, x:x2]
    ls.append(ROI.copy())
    nc += 1
print(nc)

100
96


In [327]:
len(ls)

196

In [328]:
path = '02'
rows_num = 49
os.makedirs(path, exist_ok=True)
rmtree(path)
os.makedirs(path, exist_ok=True)
nc=0
for i in range(rows_num):
    for j in range(4):
        Image.fromarray(ls[nc]).save(f"{path}/{names[196 + i * 1 + j * 49].split('/')[-1]}")
        nc+=1

In [233]:
def draw_bounding_box_on_image_array(image,
                                     xmin,
                                     ymin,
                                     xmax,
                                     ymax,
                                     color='red',
                                     thickness=4,
                                     display_str_list=(),
                                     use_normalized_coordinates=True):
  image_pil = Image.fromarray(np.uint8(image)).convert('RGB')
  draw_bounding_box_on_image(image_pil, ymin, xmin, ymax, xmax, color,
                             thickness, display_str_list,
                             use_normalized_coordinates)
  np.copyto(image, np.array(image_pil))


def draw_bounding_box_on_image(image,
                               xmin,
                               ymin,
                               xmax,
                               ymax,
                               color='red',
                               thickness=4,
                               display_str_list=(),
                               use_normalized_coordinates=False):
  draw = ImageDraw.Draw(image)
  im_width, im_height = image.size
  if use_normalized_coordinates:
    (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                  ymin * im_height, ymax * im_height)
  else:
    (left, right, top, bottom) = (xmin, xmax, ymin, ymax)
  draw.line([(left, top), (left, bottom), (right, bottom),
             (right, top), (left, top)], width=thickness, fill=color)
  try:
    font = ImageFont.truetype('arial.ttf', 24)
  except IOError:
    font = ImageFont.load_default()

#   print((left, right, top, bottom))
  # If the total height of the display strings added to the top of the bounding
  # box exceeds the top of the image, stack the strings below the bounding box
  # instead of above.
  display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
  # Each display_str has a top and bottom margin of 0.05x.
  total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

  if top > total_display_str_height:
    text_bottom = top
  else:
    text_bottom = bottom + total_display_str_height
  # Reverse list and print from bottom to top.
  for display_str in display_str_list[::-1]:
    text_width, text_height = font.getsize(display_str)
    margin = np.ceil(0.05 * text_height)
    draw.rectangle(
        [(left, text_bottom - text_height - 2 * margin), (left + text_width,
                                                          text_bottom)],
        fill=color)
    draw.text(
        (left + margin, text_bottom - text_height - margin),
        display_str,
        fill='black',
        font=font)
    text_bottom -= text_height - 2 * margin


def draw_bounding_boxes_on_image_array(image,
                                       boxes,
                                       color='red',
                                       thickness=4,
                                       display_str_list_list=()):
  image_pil = Image.fromarray(image)
  draw_bounding_boxes_on_image(image_pil, boxes, color, thickness,
                               display_str_list_list)
  np.copyto(image, np.array(image_pil))
#   return image_pil


def draw_bounding_boxes_on_image(image,
                                 boxes,
                                 color='red',
                                 thickness=4,
                                 display_str_list_list=()):
  boxes_shape = boxes.shape
  if not boxes_shape:
    return
  if len(boxes_shape) != 2 or boxes_shape[1] != 4:
    raise ValueError('Input must be of size [N, 4]')
  for i in range(boxes_shape[0]):
    display_str_list = ()
    if display_str_list_list:
      display_str_list = display_str_list_list[i]
    draw_bounding_box_on_image(image, boxes[i, 0], boxes[i, 1], boxes[i, 2],
                               boxes[i, 3], color, thickness, display_str_list)

In [121]:
# import the necessary packages
import numpy as np
 
# Malisiewicz et al.
def non_max_suppression_fast(boxes, overlapThresh):
	# if there are no boxes, return an empty list
	if len(boxes) == 0:
		return []
 
	# if the bounding boxes integers, convert them to floats --
	# this is important since we'll be doing a bunch of divisions
	if boxes.dtype.kind == "i":
		boxes = boxes.astype("float")
 
	# initialize the list of picked indexes	
	pick = []
 
	# grab the coordinates of the bounding boxes
	x1 = boxes[:,0]
	y1 = boxes[:,1]
	x2 = boxes[:,2]
	y2 = boxes[:,3]
 
	# compute the area of the bounding boxes and sort the bounding
	# boxes by the bottom-right y-coordinate of the bounding box
	area = (x2 - x1 + 1) * (y2 - y1 + 1)
	idxs = np.argsort(y2)
 
	# keep looping while some indexes still remain in the indexes
	# list
	while len(idxs) > 0:
		# grab the last index in the indexes list and add the
		# index value to the list of picked indexes
		last = len(idxs) - 1
		i = idxs[last]
		pick.append(i)
 
		# find the largest (x, y) coordinates for the start of
		# the bounding box and the smallest (x, y) coordinates
		# for the end of the bounding box
		xx1 = np.maximum(x1[i], x1[idxs[:last]])
		yy1 = np.maximum(y1[i], y1[idxs[:last]])
		xx2 = np.minimum(x2[i], x2[idxs[:last]])
		yy2 = np.minimum(y2[i], y2[idxs[:last]])
 
		# compute the width and height of the bounding box
		w = np.maximum(0, xx2 - xx1 + 1)
		h = np.maximum(0, yy2 - yy1 + 1)
 
		# compute the ratio of overlap
		overlap = (w * h) / area[idxs[:last]]
 
		# delete all indexes from the index list that have
		overlaps = np.concatenate(([last],np.where(overlap > overlapThresh)[0]))
		idxs = np.delete(idxs, overlaps)
 
	# return only the bounding boxes that were picked using the
	# integer data type
	return boxes[pick].astype("int")